In [274]:
import sys
sys.path.insert(0, "/Users/jeremythaller/Documents/python-scripts/GoogleSheetsAPI")
from google_conn import *
from database_conn import BackOffice, Veritas
# import cdf as pd
import pandas as pd
bo = BackOffice()
ver = Veritas()
import plotly
import datetime as dt
from datetime import date

In [275]:
def fix_dates(date1):
    return str(dt.datetime(1899, 12, 30) + dt.timedelta(days=date1))[:10]

Pull data from the Growth Dashboard Sheet

In [276]:
GROWTH_DASH_SHEET_ID = '12f8T3_w-eO4vfT3UkoD2ErSCbGwFnEVF44C9Ya96QnA'
dash_table = GoogleSheets(spreadsheetId=GROWTH_DASH_SHEET_ID)
result = dash_table.batchGet(ranges=['A1:O'])
data = result[0].get('values')
df = pd.DataFrame(data[1:], columns=[x.replace('\n', ' ') for x in data[0]])
df['Registration Date'] = df['Registration Date'].apply(fix_dates)
df.head()

,Registration Date,Registrations,Passed KYC,Made a Deposit,Bought Coin,Made a Swap,Took a Loan,Total KYC Incomplete,Completed Authentification,Verified Indentity,Confirmed Location,Reg to KYC Passed %,Reg to Deposit %,KYC to Deposit %,BTC Price
0,2021-01-01,2257,1651,1049,255,33,90,606,304,190,161,0.731502,0.464776,0.635373,29344.673831
1,2021-01-02,3922,2562,1474,419,64,119,1360,690,458,389,0.653238,0.375829,0.575332,32072.078133
2,2021-01-03,4621,2969,1619,465,44,116,1652,841,589,503,0.642502,0.350357,0.545301,33048.027784
3,2021-01-04,3285,2325,1362,316,42,116,960,492,322,274,0.707763,0.414612,0.585806,32084.607546
4,2021-01-05,3197,2297,1354,309,43,128,900,479,314,266,0.718486,0.423522,0.589465,34105.463383


In [277]:
event_table = GoogleSheets(spreadsheetId=GROWTH_DASH_SHEET_ID)
result = event_table.batchGet(ranges=['Events Timeline!A1:C'])
data = result[0].get('values')
df2 = pd.DataFrame(data[1:], columns=[x.replace('\n', ' ') for x in data[0]])
df2['Date'] = df2['Date'].apply(fix_dates)
df2.head()

,Date,Internal Event,External Event
0,2021-01-01,BTC Rate: From 6.39% to 6.2 up to 2 BTC and 4....,None
1,2021-01-02,,CMC - Mashinsky Top 10
2,2021-01-03,None,None
3,2021-01-04,Alex Mashinsky & Charles Hoskinson,None
4,2021-01-05,None,None


In [278]:
# Internal Events
df_temp = df2[~df2['Internal Event'].isna()]
df_events = df[df["Registration Date"].isin(df_temp.Date)]
df_events = df_events.merge(df_temp, left_on="Registration Date", right_on="Date", how='left')

In [279]:
df_events

,Registration Date,Registrations,Passed KYC,Made a Deposit,Bought Coin,Made a Swap,Took a Loan,Total KYC Incomplete,Completed Authentification,Verified Indentity,Confirmed Location,Reg to KYC Passed %,Reg to Deposit %,KYC to Deposit %,BTC Price,Date,Internal Event,External Event
0,2021-01-01,2257,1651,1049,255,33,90,606,304,190,161,0.731502,0.464776,0.635373,29344.673831,2021-01-01,BTC Rate: From 6.39% to 6.2 up to 2 BTC and 4....,None
1,2021-01-02,3922,2562,1474,419,64,119,1360,690,458,389,0.653238,0.375829,0.575332,32072.078133,2021-01-02,,CMC - Mashinsky Top 10
2,2021-01-04,3285,2325,1362,316,42,116,960,492,322,274,0.707763,0.414612,0.585806,32084.607546,2021-01-04,Alex Mashinsky & Charles Hoskinson,None
3,2021-01-09,2953,2109,1218,290,45,106,844,448,265,236,0.714189,0.412462,0.577525,40397.522570,2021-01-09,,CryptoWise Video
4,2021-01-15,2107,1406,819,190,25,66,701,360,226,197,0.667299,0.388704,0.582504,36705.814268,2021-01-15,BTC rate to 4.7% from 4.5% for more than 2 BTC,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,2022-02-11,2515,1203,541,51,2,7,1312,887,438,342,0.478330,0.215109,0.449709,42434.585857,2022-02-11,Solana + Songbird,None
118,2022-02-12,2285,1107,523,70,7,8,1178,777,380,316,0.484464,0.228884,0.472448,42189.242245,2022-02-12,,BTC $45K
119,2022-02-14,2319,1174,566,64,12,13,1145,729,342,274,0.506253,0.244071,0.482112,42739.422116,2022-02-14,,BlockFi $100M SEC fine
120,2022-02-22,1870,1020,443,41,4,7,850,607,304,262,0.545455,0.236898,0.434314,38362.142544,2022-02-22,"CelsiusX launches, CEL Promo (Non-US)",None


In [280]:
# External Events
df_temp = df2[~df2['External Event'].isna()]
df_events_ex = df[df["Registration Date"].isin(df_temp.Date)]
df_events_ex = df_events_ex.merge(df_temp, left_on="Registration Date", right_on="Date", how='left')
df_events_ex

,Registration Date,Registrations,Passed KYC,Made a Deposit,Bought Coin,Made a Swap,Took a Loan,Total KYC Incomplete,Completed Authentification,Verified Indentity,Confirmed Location,Reg to KYC Passed %,Reg to Deposit %,KYC to Deposit %,BTC Price,Date,Internal Event,External Event
0,2021-01-02,3922,2562,1474,419,64,119,1360,690,458,389,0.653238,0.375829,0.575332,32072.078133,2021-01-02,,CMC - Mashinsky Top 10
1,2021-01-09,2953,2109,1218,290,45,106,844,448,265,236,0.714189,0.412462,0.577525,40397.522570,2021-01-09,,CryptoWise Video
2,2021-01-28,1727,1218,669,140,27,51,509,214,118,98,0.705269,0.387377,0.549261,33165.475219,2021-01-28,,Coinsider Video
3,2021-02-02,2454,1694,1018,206,33,70,760,309,187,155,0.690302,0.414833,0.600945,35410.054050,2021-02-02,,Top 100 People in Blockchain
4,2021-02-09,4125,3024,1658,339,44,131,1101,438,286,252,0.733091,0.401939,0.548280,46398.460101,2021-02-09,,Elon Musk BTC pump +10K in 1 day
5,2021-02-13,3715,2783,1626,332,41,100,932,430,295,260,0.749125,0.437685,0.584262,46938.567872,2021-02-13,,Celsius Network Review - Wes Spencer
6,2021-04-01,3042,2118,1078,132,41,73,924,438,296,273,0.696252,0.354372,0.508971,58794.306287,2021-04-01,"ARRA Global Summit, Celsius now offers yield o...","ARRA Global Summit, BlockFi Lowers Rates"
7,2021-04-02,2889,2083,1091,134,40,96,806,404,270,247,0.721011,0.377639,0.523764,59035.727832,2021-04-02,,Best Crypto Wallet
8,2021-04-11,2940,2175,1181,166,41,63,765,344,216,184,0.739796,0.401701,0.542989,60056.725726,2021-04-11,,Miami Crypto Conf
9,2021-04-23,2514,1822,856,178,29,64,692,291,173,144,0.724741,0.340493,0.469813,51133.546342,2021-04-23,,Knowledge By Marcus


In [281]:
import plotly.express as px
import plotly.graph_objects as go

fig1 = px.line(df, x=df["Registration Date"], y=df["Registrations"]/df["BTC Price"], title='BTC Normalized Registrations')
fig1.update_traces(line=dict(color = 'rgba(50,50,50,0.2)'))

fig2 = px.scatter(df_events, x="Registration Date", y=df_events["Registrations"]/df_events["BTC Price"],  hover_data=["BTC Price", "Registrations"],
                  hover_name='Internal Event')

fig3 = px.scatter(df_events_ex, x="Registration Date", y=df_events_ex["Registrations"]/df_events_ex["BTC Price"],
                  hover_data=["BTC Price", "Registrations"], hover_name="External Event")
fig3.update_traces(marker=dict(color = 'Red'))

fig4 = go.Figure(data=fig1.data + fig2.data + fig3.data)

fig4.update_layout(title='BTC Normalized Registrations', xaxis_title="Date",
                   yaxis_title="Registrations/BTC", legend_title="Metric (click to add/remove)")
fig4.show()

In [282]:
fig4.write_html('../simplified-growth-timeline/index.html')

In [283]:
import plotly.graph_objects as go
from plotly.validators.scatter.marker import SymbolValidator
import plotly.io as pio

fig = go.Figure()

fig.add_trace(go.Scatter(x=df["Registration Date"], y=df["Registrations"]/df["BTC Price"],
                         name="<b>Normalized Registrations</b>",  hovertext=df[["Registrations", "BTC Price"]], hovertemplate=" %{y}<br><b>Regs</b>: %{hovertext[0]}<br><b>BTC Price</b>: %{hovertext[1]:$,.0f}<br>"
                         ))

fig.add_trace(go.Scatter(x=df["Registration Date"], y=df["Passed KYC"]/df["BTC Price"],
                         name="<b>Normalized KYC-Passed</b>",
                         hovertext=df["Passed KYC"], hovertemplate=" %{y}<br><b>Passed KYC</b>: %{hovertext}<br>"
                         ))

fig.add_trace(go.Scatter(x=df["Registration Date"], y=df["Made a Deposit"]/df["BTC Price"],
                         name="<b>Normalized Deposited</b>",
                         hovertext=df["Made a Deposit"], hovertemplate=" %{y}<br><b>Made a Deposit</b>: %{hovertext}<br>"
                         ))

fig.add_trace(go.Scatter(x=df["Registration Date"], y=df["Reg to KYC Passed %"],
                         name="<b>Reg -> KYC Pass %</b>", visible='legendonly',
                         hovertext=df["Reg to KYC Passed %"], hovertemplate=" %{y:.1%}<br>"
                         ))

# ----- circle version on regs --------
# fig.add_trace(go.Scatter(x=df_events["Registration Date"], y=df_events["Registrations"]/df_events["BTC Price"],
#                          name="Event", mode='markers', marker=dict(size=12, line=dict(width=2, color='DarkSlateGrey')),
#                          hovertext=df_events["Internal Event"]
#                          ))

fig.add_trace(go.Scatter(x=df_events["Registration Date"], y=[.05 for _ in range(df_events["Registration Date"].shape[0])],
                         name="<b>Internal Event</b>", mode='markers', marker_symbol="line-ns", marker=dict(size=100000, line=dict(width=2, color='rgba(50, 56, 68, 0.5)')),
                         hovertext=df_events["Internal Event"], hovertemplate="<i>%{hovertext}</i><br>", hoveron='points', visible='legendonly'
                         ))

fig.add_trace(go.Scatter(x=df_events_ex["Registration Date"], y=[.05 for _ in range(df_events_ex["Registration Date"].shape[0])],
                         name="<b>External Event</b>", mode='markers', marker_symbol="line-ns", marker=dict(size=100000, line=dict(width=2, color='rgba(50, 56, 68, 0.5)')),
                         hovertext=df_events_ex["External Event"], hovertemplate="<i>%{hovertext}</i><br>", hoveron='points', visible='legendonly'
                         ))

fig.update_layout(
    title="BTC Normalized Registrations",
    xaxis_title="Date",
    yaxis_title="",
    legend_title="Metric (click to add/remove)",
    font=dict(size=18),
    hovermode="x unified",
    hoverdistance=1,
    xaxis_range=['2021-01-01', df["Registration Date"][df["Registration Date"].shape[0]-1]],
    #template='plotly_dark'
)

# hovertemplate="%{x}%{hover.text}",
# df_events["Registrations"]/df_events["BTC Price"]

fig.show()

In [284]:
# write the file
fig.write_html("index.html")

In [285]:
df2

,Date,Internal Event,External Event
0,2021-01-01,BTC Rate: From 6.39% to 6.2 up to 2 BTC and 4....,None
1,2021-01-02,,CMC - Mashinsky Top 10
2,2021-01-03,None,None
3,2021-01-04,Alex Mashinsky & Charles Hoskinson,None
4,2021-01-05,None,None
...,...,...,...
420,2022-02-25,None,None
421,2022-02-26,None,None
422,2022-02-27,None,None
423,2022-02-28,None,None


In [286]:
import numpy as np
df2["External Event bool"] = df2["External Event"].apply(lambda x: np.NAN if x in (None, '', " ", "None") else .05)
df2["Internal Event bool"] = df2["Internal Event"].apply(lambda x: np.NAN if x in (None, '', " ", "None") else .05)


In [287]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=df["Registration Date"], y=df["Registrations"]/df["BTC Price"],
                         name="<b>Normalized Registrations</b>",  hovertext=df[["Registrations", "BTC Price"]], hovertemplate=" %{y}<br><b>Regs</b>: %{hovertext[0]}<br><b>BTC Price</b>: %{hovertext[1]:$,.0f}<br>"
                         ))

fig.add_trace(go.Scatter(x=df["Registration Date"], y=df["Passed KYC"]/df["BTC Price"],
                         name="<b>Normalized KYC-Passed</b>",
                         hovertext=df["Passed KYC"], hovertemplate=" %{y}<br><b>Passed KYC</b>: %{hovertext}<br>"
                         ))

fig.add_trace(go.Scatter(x=df["Registration Date"], y=df["Made a Deposit"]/df["BTC Price"],
                         name="<b>Normalized Deposited</b>",
                         hovertext=df["Made a Deposit"], hovertemplate=" %{y}<br><b>Made a Deposit</b>: %{hovertext}<br>"
                         ))

fig.add_trace(go.Scatter(x=df["Registration Date"], y=df["Reg to KYC Passed %"],
                         name="<b>Reg → KYC Pass %</b>", visible='legendonly',
                         hovertext=df["Reg to KYC Passed %"], hovertemplate=" %{y:.1%}<br>"
                         ))

fig.add_trace(go.Scatter(x=df["Registration Date"], y=df["KYC to Deposit %"],
                         name="<b>KYC → Deposit %</b>", visible='legendonly',
                         hovertext=df["KYC to Deposit %"], hovertemplate=" %{y:.1%}<br>"
                         ))

fig.add_trace(go.Scatter(x=df["Registration Date"], y=df["Reg to Deposit %"],
                         name="<b>Reg → Deposit %</b>", visible='legendonly',
                         hovertext=df["Reg to Deposit %"], hovertemplate=" %{y:.1%}<br>"
                         ))




# ----- circle version on regs --------
# fig.add_trace(go.Scatter(x=df_events["Registration Date"], y=df_events["Registrations"]/df_events["BTC Price"],
#                          name="Event", mode='markers', marker=dict(size=12, line=dict(width=2, color='DarkSlateGrey')),
#                          hovertext=df_events["Event"]
#                          ))

fig.add_trace(go.Scatter(x=df2["Date"], y=df2["Internal Event bool"],
                         name="<b>Internal Event</b>", mode='markers', marker_symbol="line-ns", marker=dict(size=100000, line=dict(width=2, color='rgba(222, 76, 170, 0.5)')),
                         hovertext=df2["Internal Event"], hovertemplate="<i>%{hovertext}</i><br>", hoveron='points', visible='legendonly'
                         ))


fig.add_trace(go.Scatter(x=df2["Date"], y=df2["External Event bool"],
                         name="<b>External Event</b>", mode='markers', marker_symbol="line-ns", marker=dict(size=100000, line=dict(width=2, color='rgba(250, 178, 56, 0.5)')),
                         hovertext=df2["External Event"], hovertemplate="<i>%{hovertext}</i><br>", hoveron='points', visible='legendonly'
                         ))

fig.update_layout(
    title="KPI Tracking and Events Timeline<br><br>",
    xaxis_title="Date",
    yaxis_title="",
    legend_title="Metric (click to add/remove)",
    font=dict(size=18),
    hovermode="x unified",
    hoverdistance=1,
    xaxis_range=['2020-12-31', df["Registration Date"][df["Registration Date"].shape[0]-1]],
    #template='plotly_dark'
)


# hovertemplate="%{x}%{hover.text}",
# df_events["Registrations"]/df_events["BTC Price"]
fig.update_xaxes(
    rangeslider_visible=False,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.update_layout(margin=dict(pad=10))
fig.show()

In [288]:
# write the file
fig.write_html("index.html")

In [289]:
df2

,Date,Internal Event,External Event,External Event bool,Internal Event bool
0,2021-01-01,BTC Rate: From 6.39% to 6.2 up to 2 BTC and 4....,None,NaN,0.05
1,2021-01-02,,CMC - Mashinsky Top 10,0.05,NaN
2,2021-01-03,None,None,NaN,NaN
3,2021-01-04,Alex Mashinsky & Charles Hoskinson,None,NaN,0.05
4,2021-01-05,None,None,NaN,NaN
...,...,...,...,...,...
420,2022-02-25,None,None,NaN,NaN
421,2022-02-26,None,None,NaN,NaN
422,2022-02-27,None,None,NaN,NaN
423,2022-02-28,None,None,NaN,NaN
